In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
import shap
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, StratifiedKFold, cross_validate, cross_val_score, cross_val_predict
from sklearn.metrics import matthews_corrcoef, confusion_matrix, roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.combine import SMOTETomek
from imblearn.pipeline import Pipeline
from sklearn.metrics import recall_score, accuracy_score
from sklearn.impute import KNNImputer, SimpleImputer

In [2]:
random_state = 7
cv_method = StratifiedKFold(n_splits=5)

# Training of tsfresh-selected Training Max 1 years and eGFR 6 times

In [3]:
df = pd.read_csv('/home/jupyter-dchristiadi85/PhD Project 1/data/train_selected_1_6.gz')
df.shape

(4633, 23639)

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4633 entries, 0 to 4632
Columns: 23639 entries, id to cat8
dtypes: float64(23617), int64(2), object(20)
memory usage: 835.6+ MB


In [5]:
df['aki_1'].fillna(value=0, inplace=True)
df['aki_2'].fillna(value=0, inplace=True)
df['aki_3'].fillna(value=0, inplace=True)

In [6]:
df['gn'].value_counts()

no                 4220
igan                167
anca                 76
lupus nephritis      39
membranous           32
unspecific           28
fsgs                 27
mcd                  21
mp/mcgn              16
fgn/itg               5
pign                  2
Name: gn, dtype: int64

In [7]:
df.loc[df.gn == 'pign', 'gn'] = 'unspecific'
df.loc[df.gn == 'fgn/itg', 'gn'] = 'unspecific'

In [8]:
percent_missing = df.isnull().sum()*100 / len(df)
missing_df = pd.DataFrame({'column_name': df.columns, 'percent_miss': percent_missing})

In [9]:
missing_df.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
"mch__fft_coefficient__attr_""""""""""""""""angle""""""""""""""""__coeff_67","mch__fft_coefficient__attr_""""""""""""""""angle""""""""""""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_67","platelet__fft_coefficient__attr_""""""""""""""""abs""""""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_73","platelet__fft_coefficient__attr_""""""""""""""""abs""""""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_72","platelet__fft_coefficient__attr_""""""""""""""""abs""""""...",100.0
"platelet__fft_coefficient__attr_""""""""""""""""abs""""""""""""""""__coeff_71","platelet__fft_coefficient__attr_""""""""""""""""abs""""""...",100.0
...,...,...
htn,htn,0.0
aki_3,aki_3,0.0
aki_2,aki_2,0.0
aki_1,aki_1,0.0


### Number of patients reaching ESKD within training period 

In [10]:
df['cat1'].value_counts()

non_eskd    3522
eskd        1111
Name: cat1, dtype: int64

In [11]:
df.set_index('id', inplace=True)

In [12]:
def to_category(df):
    cols = df.select_dtypes(include='object').columns
    for col in cols:
        df[col] = df[col].astype('category')
    return df

def drop_missing(df):
    threshold = len(df)*0.6
    df.dropna(axis=1, thresh=threshold, inplace=True)
    return df

def copy_df(df):
    return df.copy()

In [13]:
df_cleaned = (df.pipe(copy_df).pipe(drop_missing).pipe(to_category))
df_cleaned.shape

(4633, 5152)

In [14]:
df_rm_intrain = df_cleaned.query("cat1 == 'non_eskd'")
df_rm_intrain.shape

(3522, 5152)

In [15]:
df_rm_intrain.to_csv('../data/cleaned60_selected_1_6.gz', index=True, compression='gzip')

In [16]:
df_rm_intrain.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3522 entries, 3916 to 23864
Columns: 5152 entries, mchc__variance_larger_than_standard_deviation to cat8
dtypes: category(20), float64(5131), int64(1)
memory usage: 138.0 MB


In [112]:
df_rm_intrain = (df_rm_intrain.pipe(copy_df).pipe(to_category))
df_rm_intrain.shape

(4409, 8985)

### Number of patients reaching ESKD in at 1y post-training 

In [17]:
df_rm_intrain['cat2'].value_counts()

non_eskd    3396
eskd         126
Name: cat2, dtype: int64

### Number of patients reaching ESKD in 2y post-training

In [18]:
df_rm_intrain['cat3'].value_counts()

non_eskd    3289
eskd         233
Name: cat3, dtype: int64

### Number of patients reaching ESKD in 5y post-training

In [19]:
df_rm_intrain['cat6'].value_counts()

non_eskd    3033
eskd         489
Name: cat6, dtype: int64

In [20]:
percent_missing = df_rm_intrain.isnull().sum()*100 / len(df_rm_intrain)
missing_df_rm_intrain = pd.DataFrame({'column_name': df_rm_intrain.columns, 'percent_miss': percent_missing})

In [21]:
missing_df_rm_intrain.sort_values(by='percent_miss', ascending=False)

,column_name,percent_miss
creatinine__autocorrelation__lag_2,creatinine__autocorrelation__lag_2,48.409994
"creatinine__cwt_coefficients__coeff_2__w_5__widths_(2, 5, 10, 20)",creatinine__cwt_coefficients__coeff_2__w_5__wi...,48.211244
creatinine__skewness,creatinine__skewness,48.211244
"creatinine__cwt_coefficients__coeff_2__w_2__widths_(2, 5, 10, 20)",creatinine__cwt_coefficients__coeff_2__w_2__wi...,48.211244
"creatinine__cwt_coefficients__coeff_2__w_20__widths_(2, 5, 10, 20)",creatinine__cwt_coefficients__coeff_2__w_20__w...,48.211244
...,...,...
dkd,dkd,0.000000
htn,htn,0.000000
aki_3,aki_3,0.000000
aki_2,aki_2,0.000000


## Features and Target Input for Training 

In [22]:
dropped_cols = list(df_rm_intrain.columns[-18:])
dropped_cols.remove('age.init')
dropped_cols

['egfr.y',
 'cat0.5',
 'cat1',
 'cat1.5',
 'cat2',
 'cat2.5',
 'cat3',
 'cat3.5',
 'cat4',
 'cat4.5',
 'cat5',
 'cat5.5',
 'cat6',
 'cat6.5',
 'cat7',
 'cat7.5',
 'cat8']

In [23]:
X = df_rm_intrain.drop(dropped_cols,axis=1).copy()
y = df_rm_intrain['cat6']
X.shape, y.shape

((3522, 5135), (3522,))

In [24]:
y_mapped = y.map({'non_eskd':'no', 'eskd':'yes'})
y_mapped

id
3916      no
3918      no
3921      no
3924      no
3930      no
        ... 
23753    yes
23760     no
23771     no
23780     no
23864     no
Name: cat6, Length: 3522, dtype: category
Categories (2, object): ['yes', 'no']

In [25]:
X_train, X_test, y_train, y_test = train_test_split(X, y_mapped, test_size=0.3, stratify=y_mapped, random_state=random_state)
X_train.shape, X_test.shape, y_train.shape, y_test.shape

((2465, 5135), (1057, 5135), (2465,), (1057,))

In [26]:
lab_encode = LabelEncoder()
y_train = lab_encode.fit_transform(y_train)
y_test = lab_encode.transform(y_test)
y_train.shape, y_test.shape

((2465,), (1057,))

In [27]:
unique_values, counts = np.unique(y_train, return_counts=True)
ratio = dict(zip(unique_values, counts))
ratio

{0: 2123, 1: 342}

In [28]:
imbalance_ratio = ratio[0] / ratio[1]
imbalance_ratio

6.207602339181286

In [29]:
category_cols = list(X_train.select_dtypes(include='category').columns)
numeric_cols = list(X_train.select_dtypes(include=['number']).columns)
one_hot = OneHotEncoder(handle_unknown='ignore')
num_imputer = KNNImputer(n_neighbors=5)
len(category_cols), len(numeric_cols)

(4, 5131)

In [30]:
preprocess = ColumnTransformer([('numerical', num_imputer, numeric_cols), ('cat_encoder', one_hot, category_cols)], remainder='passthrough')
X_train_processed = preprocess.fit_transform(X_train)
X_train_processed.shape

(2465, 5146)

In [31]:
X_test_processed = preprocess.transform(X_test)
X_test_processed.shape

(1057, 5146)

In [32]:
xgb_class = xgb.XGBClassifier(n_jobs=-1, random_state = random_state, n_estimators=1000, use_label_encoder=False, eval_metric='logloss', scale_pos_weight=imbalance_ratio)
sampler = SMOTETomek(sampling_strategy=0.5, random_state=random_state)
steps = [('resampling', sampler), ('model', xgb_class)]
pipeline = Pipeline(steps=steps)

In [33]:
pipeline.fit(X_train_processed, y_train)

Pipeline(steps=[('resampling',
                 SMOTETomek(random_state=7, sampling_strategy=0.5)),
                ('model',
                 XGBClassifier(base_score=0.5, booster='gbtree',
                               colsample_bylevel=1, colsample_bynode=1,
                               colsample_bytree=1, eval_metric='logloss',
                               gamma=0, gpu_id=-1, importance_type='gain',
                               interaction_constraints='',
                               learning_rate=0.300000012, max_delta_step=0,
                               max_depth=6, min_child_weight=1, missing=nan,
                               monotone_constraints='()', n_estimators=1000,
                               n_jobs=-1, num_parallel_tree=1, random_state=7,
                               reg_alpha=0, reg_lambda=1,
                               scale_pos_weight=6.207602339181286, subsample=1,
                               tree_method='exact', use_label_encoder=False,
     

In [34]:
y_pred = pipeline.predict(X_test_processed)

In [35]:
confusion_matrix(y_true=y_test, y_pred=y_pred)

array([[866,  44],
       [ 82,  65]])

In [36]:
roc_auc_score(y_test, y_pred)

0.6969126111983255

In [37]:
print(classification_report(y_true=y_test, y_pred=y_pred))

              precision    recall  f1-score   support

           0       0.91      0.95      0.93       910
           1       0.60      0.44      0.51       147

    accuracy                           0.88      1057
   macro avg       0.75      0.70      0.72      1057
weighted avg       0.87      0.88      0.87      1057



In [38]:
print(matthews_corrcoef(y_test, y_pred))

0.4480902279470589


In [39]:
importance = pipeline.named_steps['model'].feature_importances_
importance.shape, type(importance)

((5146,), numpy.ndarray)

In [40]:
category_label = list(preprocess.named_transformers_['cat_encoder'].get_feature_names(category_cols))
feature_label = numeric_cols + category_label
len(feature_label)

5146

In [41]:
pd.set_option('display.max_rows', 400)
eval_df = pd.DataFrame({'label': feature_label, 'importance_value': importance})
eval_df.sort_values(by='importance_value', ascending=False).head(100)

,label,importance_value
3569,creatinine__quantile__q_0.3,0.109020
2675,"albumin__change_quantiles__f_agg_""""""""""""""""mean""...",0.022457
3517,creatinine__maximum,0.019514
3003,alkphos__lempel_ziv_complexity__bins_10,0.017837
4675,mch__sum_values,0.015455
1058,"platelet__change_quantiles__f_agg_""""""""""""""""mean...",0.014196
2747,albumin__energy_ratio_by_chunks__num_segments_...,0.010542
2810,alkphos__cid_ce__normalize_True,0.010034
4673,mch__has_duplicate_min,0.009794
3349,caphos_product__index_mass_quantile__q_0.2,0.009780
